In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model

import datetime
import os

Using TensorFlow backend.


In [2]:
data = pd.read_csv('dataset/PM_2.5_dataset.csv', index_col=0)
data.head()

,PM,temprature,humidity,max wind speed,wind direction
Date,,,,,
2009-01-01,43.739130,-5.8,45.8,4.5,290.0
2009-01-02,45.750000,-2.8,51.5,3.7,230.0
2009-01-03,46.833333,-1.0,48.3,3.9,250.0
2009-01-04,40.083333,0.4,44.3,4.3,290.0
2009-01-05,58.541667,-1.6,53.1,4.5,270.0


In [15]:
PM = data['PM'].values

seq_len = 30
sequence_length = seq_len + 1

result = []
for index in range(len(PM) - sequence_length):
    result.append(PM[index: index + sequence_length])
    
result_in = np.array(result)

result의 결과를 확인하면 다음 array마다 0번째 요소가 지워지고 -1번째 요소로 새로운 데이터(mid_price)가 입력되는 것을 확인할 수 있다.

In [22]:

def normalize_windows(data):
    normalized_data = []
    first_data = []
    for window in data:
        first_data.append(window[0])
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
    return np.array(normalized_data), first_data

norm_result, first_data = normalize_windows(result)

In [23]:
first_data

[43.73913043,
 45.75,
 46.83333333,
 40.08333333,
 58.54166667,
 96.0,
 83.125,
 72.20833333,
 65.25,
 54.08333333,
 41.5,
 52.91666667,
 42.54166667,
 62.41666667,
 38.58333333,
 53.79166667,
 94.125,
 108.54166670000001,
 81.79166667,
 48.5,
 56.125,
 61.0,
 73.875,
 44.33333333,
 46.95833333,
 33.08333333,
 45.54166667,
 45.58333333,
 58.66666667,
 38.70833333,
 21.45833333,
 27.04166667,
 48.75,
 75.08333333,
 104.16666670000001,
 103.41666670000001,
 142.66666669999998,
 141.625,
 103.08333329999999,
 52.45833333,
 62.25,
 170.45833330000002,
 159.54166669999998,
 46.70833333,
 74.45833333,
 54.66666667,
 33.08333333,
 32.375,
 34.75,
 55.91666667,
 237.33333330000002,
 94.625,
 78.79166667,
 57.875,
 42.58333333,
 63.5,
 46.0,
 52.66666667,
 62.41666667,
 53.66666667,
 40.20833333,
 35.95833333,
 82.75,
 59.875,
 59.08333333,
 33.70833333,
 36.45833333,
 54.625,
 53.08333333,
 40.79166667,
 46.125,
 30.25,
 91.125,
 64.45833333,
 151.29166669999998,
 252.0,
 40.375,
 38.375,
 31.

In [20]:
# train_cut = -90 # 마지막 3달의 데이터를 test데이터로 한다
train_cut = int(round(result_in.shape[0] * 0.70))

valid_cut = -365
        
train = result_in[:train_cut, :]
np.random.shuffle(train)

valid = result_in[train_cut:valid_cut, :]

test = result_in[valid_cut:, :]

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:,-1]
y_train = np.reshape(y_train, (y_train.shape[0], 1))

x_valid = valid[:, :-1]
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
y_valid = valid[:,-1]
y_valid = np.reshape(y_valid, (y_valid.shape[0], 1))

x_test = result_in[valid_cut:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result_in[valid_cut:,-1]
y_test = np.reshape(y_test, (y_test.shape[0], 1))

print(x_train.shape, x_valid.shape, x_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(2535, 30, 1) (721, 30, 1) (365, 30, 1)
(2535, 1) (721, 1) (365, 1)


MODEL_SAVE_FOLBER_PATH = './model/checkpoint/'
if not os.path.exists(MODEL_SAVE_FOLBER_PATH):
    os.mkdir(MODEL_SAVE_FOLBER_PATH)
    
model_path = MODEL_SAVE_FOLBER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'

# 체크포인트 생성 코드
cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',verbose=1, save_best_only=True, mode='auto')

In [ ]:
# 모델 불러오기
# model = load_model('bitcoin_prediction_model.h5')

In [8]:
model = Sequential()

for i in range (4):
    model.add(LSTM(30, return_sequences=True, input_shape=(30, 1)))   

model.add(LSTM(30, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()

W1022 19:37:22.247427 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1022 19:37:22.264422 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1022 19:37:22.267420 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1022 19:37:22.935215 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 30)            3840      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 33,151
Trainable params: 33,151
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=30, epochs=30)

W1022 19:37:33.133936 10836 deprecation.py:323] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1022 19:37:35.313248 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1022 19:37:35.317251 10836 deprecation_wrapper.py:119] From D:\_setup\anaconda3\envs\py3.7\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 2535 samples, validate on 365 samples
Epoch 1/30
2535/2535 [==============================] - 23s 9ms/step - loss: 2544.3666 - val_loss: 1095.9068
Epoch 2/30
2535/2535 [==============================] - 19s 8ms/step - loss: 2266.6512 - val_loss: 964.3442
Epoch 3/30
2535/2535 [==============================] - 19s 8ms/step - loss: 2088.2022 - val_loss: 847.4939
Epoch 4/30
2535/2535 [==============================] - 19s 7ms/step - loss: 1919.2346 - val_loss: 743.4812
Epoch 5/30
2535/2535 [==============================] - 19s 7ms/step - loss: 1768.1497 - val_loss: 653.9122
Epoch 6/30
2535/2535 [==============================] - 19s 7ms/step - loss: 1628.6910 - val_loss: 577.0858
Epoch 7/30
2535/2535 [==============================] - 20s 8ms/step - loss: 1505.2943 - val_loss: 513.8228
Epoch 8/30
2535/2535 [==============================] - 20s 8ms/step - loss: 1393.9837 - val_loss: 464.3267
Epoch 9/30
2535/2535 [==============================] - 19s 8ms/step - loss: 1296.1397 -

In [ ]:
from keras.models import load_model

# 모델 저장
# model. save('ether_prediction_model_achi_0.h5')
model. save('model/PM_prediction_model_achi_0_1.h5')


In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white', figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(y_test, label='True', marker='.')
ax.plot(pred, label='Prediction', marker='^')
ax.legend()
ax.grid(linestyle='-', linewidth=1,)
plt.ylim([0, 200])
plt.show()

In [12]:
pred_error = abs((pred-y_test) / y_test)

total_error_rate = sum(pred_error)/len(pred_error)
print("Total error rate: ", total_error_rate)

Total error rate:  [1.64285813]
